In [ ]:
import pandas as pd
import io
import re
import matplotlib.pyplot as plt

directory = '../tempFolder'
fileName  = 'allE_all_64pts.txt'
with open(directory + '/' + fileName, 'r') as f:
    data = f.readlines()

data[4] = data[4].replace('%','')
data[4] = data[4].replace(' (cm)','(cm)')
data[4] = data[4].replace(' (MHz)','(MHz)')
data[4] = data[4].replace('abs(emw.Ex) (V/m), Point: (', ' Ex_atPoint(')
data[4] = data[4].replace('abs(emw.Ey) (V/m), Point: (', ' Ey_atPoint(')
data[4] = data[4].replace('abs(emw.Ez) (V/m), Point: (', ' Ez_atPoint(')
data[4] = data[4].replace('abs(emw.normE) (V/m), Point: (', ' Enorm_atPoint(')
data[4] = data[4].replace(', ',',')

#data[4] = data[4].replace('                    ',')')
#data[4] = data[4].replace(') (',')(')
#print(data[4])
header = data[4].split()

#print(data[4])



# read the file and skip the first 5 rows
df = pd.read_csv(directory + '/' + fileName, delim_whitespace=True, skiprows=5, header=None)
df.columns = header



# group the data by frequency and calculate the mean of each E_i, ignoring NANs
df = df.groupby('freq(MHz)').mean()

%matplotlib widget
filter_colX = [col for col in df if col.startswith('Ex')]
filter_colY = [col for col in df if col.startswith('Ey')]
filter_colZ = [col for col in df if col.startswith('Ez')]
df[filter_colX].mean(axis=1).plot(label='x')
df[filter_colY].mean(axis=1).plot(label='y')
df[filter_colZ].mean(axis=1).plot(label='z')
plt.legend()

# write the results to a new CSV file
#df.to_csv('output2.csv')

df.head()

In [ ]:
df['filter_colX']

In [ ]:
import numpy as np
def makeDf(directory, nFreqs):
        #read LPV
        df = pd.read_csv(directory+'/LPV.txt',
                skiprows=5,
                header=None, 
                names=['X', 'Y', 'Z', 'Freq', 'LPV'],
                delimiter=' ',
                skipinitialspace=True)

        E_near_x = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=4)
        E_near_y = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=5)
        E_near_z = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=6)
        E_near_norm = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=7)

        E_all_x = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=4)
        E_all_y = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=5)
        E_all_z = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=6)
        E_all_norm = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=7)

        nPoints = df.shape[0]
        nAntPositions = nPoints//nFreqs
        positions = [x for x in range(nAntPositions)]
        posList = np.repeat(positions, nFreqs)
        freqs = np.asarray(df['Freq'][0:nFreqs])

        df['Position'] = posList
#        cols = df.columns.tolist()
#        cols = cols[-1:] + cols[:-1]
#        df = df[cols]

        df['E_near_X'] = E_near_x
        df['E_near_Y'] = E_near_y
        df['E_near_Z'] = E_near_z
        df['E_near_Norm'] = E_near_norm
        df['E_all_X'] = E_all_x
        df['E_all_Y'] = E_all_y
        df['E_all_Z'] = E_all_z
        df['E_all_Norm'] = E_all_norm

        xpos = np.asarray(df.groupby('Position').mean()['X'])
        ypos = np.asarray(df.groupby('Position').mean()['Y'])
        zpos = np.asarray(df.groupby('Position').mean()['Z'])


        #set order of columns
        cols = ['Position',
                'Freq', 
                'LPV',
                'E_near_Norm',
                'E_all_Norm',
                'E_near_X',
                'E_near_Y',
                'E_near_Z',
                'E_all_X',
                'E_all_Y',
                'E_all_Z',
                'X',
                'Y',
                'Z']
        df = df[cols]

        #Print info
        print(directory)
        print(nAntPositions, " Antenna Positions :")
        print(np.asarray([positions,xpos, ypos, zpos, ]).T)
        print('Freq (start stop number step) :', freqs[0], ',', freqs[-1],',', len(freqs),',', freqs[1]-freqs[0])
        print('\n')
        return df, freqs, nAntPositions



In [ ]:
df, freqs, nAntPositoins=makeDf('./', 220)

In [ ]:
df

In [ ]:
plt.close('all')
(df.groupby('Freq').mean()['LPV'])